In [1]:
import os

cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/tarasbohdan/Desktop/Experiments-AAAI-24


In [2]:
%matplotlib inline

import sys
sys.path.append("../")
import copy

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import numpy as np


pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [3]:
class Config:
  file_name = './data/student_performance/student-mat.csv'

In [4]:
from notebooks.mult_repair_levels_student_performance.StudentPerformanceDataset import StudentPerformanceDataset

data_loader = StudentPerformanceDataset(Config.file_name)
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,Mjob,Fjob,reason,guardian,sex
0,18,4,4,2,2,0,4,3,4,1,1,3,6,at_home,teacher,course,mother,F
1,17,1,1,1,2,0,5,3,3,1,1,3,4,at_home,other,course,father,F
2,15,1,1,1,2,3,4,3,2,2,3,3,10,at_home,other,other,mother,F
3,15,4,2,1,3,0,3,2,2,1,1,5,2,health,services,home,mother,F
4,16,3,3,1,2,0,4,3,2,1,2,5,4,other,other,home,father,F


In [5]:
init_data_loader = copy.deepcopy(data_loader)

In [6]:
from virny.preprocessing import preprocess_dataset
from source.preprocessing import get_simple_preprocessor

data_loader.categorical_columns = [col for col in data_loader.categorical_columns if
                                   col not in ('sex')]
data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
data_loader.full_df = data_loader.full_df.drop(['sex'], axis=1)
data_loader.X_data = data_loader.X_data.drop(['sex'], axis=1)

# Preprocess the dataset using the defined preprocessor
column_transformer = get_simple_preprocessor(data_loader)
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, 0.3, 42)
base_flow_dataset.init_features_df = init_data_loader.full_df.drop(init_data_loader.target, axis=1,
                                                                   errors='ignore')
base_flow_dataset.X_train_val['sex_binary'] = data_loader.X_data.loc[
    base_flow_dataset.X_train_val.index, 'sex_binary']
base_flow_dataset.X_test['sex_binary'] = data_loader.X_data.loc[base_flow_dataset.X_test.index, 'sex_binary']

/var/folders/cg/l243p36n10s5v1x_vrt2wzs80000gn/T/ipykernel_24022/2079174263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
/Users/tarasbohdan/Desktop/Experiments-AAAI-24/virny_venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
from sklearn.model_selection import train_test_split

full_df = data_loader.X_data

dataset_orig_train, dataset_orig_test = train_test_split(full_df, test_size=0.3)

In [8]:
sensitive_attribute = 'sex_binary'

In [9]:
train_df = base_flow_dataset.X_train_val
train_df[base_flow_dataset.target] = base_flow_dataset.y_train_val

test_df = base_flow_dataset.X_test
test_df[base_flow_dataset.target] = base_flow_dataset.y_test

In [10]:
train_binary_dataset = BinaryLabelDataset(df=train_df,
                                              label_names=[data_loader.target],
                                              protected_attribute_names=[sensitive_attribute],
                                              favorable_label=1,
                                              unfavorable_label=0)
test_binary_dataset = BinaryLabelDataset(df=test_df,
                                         label_names=[data_loader.target],
                                         protected_attribute_names=[sensitive_attribute],
                                         favorable_label=1,
                                         unfavorable_label=0)

In [11]:
# Metric for the original dataset
privileged_groups = [{'sex_binary': 1}]
unprivileged_groups = [{'sex_binary': 0}]

metric_orig_train = BinaryLabelDatasetMetric(train_binary_dataset,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(test_binary_dataset,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.034286


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.154042


In [12]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         Az=50.0,
         verbose=1
        )
TR = TR.fit(train_binary_dataset)

step: 0, loss: 1.4647248934252552, L_x: 1.2930002362289983,  L_y: 0.6375024072866353,  L_z: 0.016285849675526598
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          160     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.46472D+00    |proj g|=  5.50848D-01
step: 250, loss: 1.451611624751507, L_x: 1.2929699849987861,  L_y: 0.6260357924639941,  L_z: 0.0162529226487505

At iterate    1    f=  1.45161D+00    |proj g|=  5.69975D-01

At iterate    2    f=  8.78809D-01    |proj g|=  5.29264D-01
step: 500, loss: 5.640273739314181, L_x: 1.1967113097398432,  L_y: 0.615568455776723,  L_z: 0.1002547634088012
step: 750, loss: 1.6614912860447353, L_x: 1.2979485766014707,  L_y: 0.6185427532961756,  L_z: 0.020599380939650896

At iterate    3    f=  6.70450D-01    |proj g|=  5.58346D-01
step: 1000, loss: 0.747391069731161, L_x: 1.2996741110165353,  L_y: 0.6214006871270966,  L_z: 0.0022598728298779793
step: 1250, loss: 

In [13]:
dataset_transf_train = TR.transform(train_binary_dataset)
dataset_transf_test = TR.transform(test_binary_dataset)

In [14]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())
metric_transf_test = BinaryLabelDatasetMetric(dataset_transf_test,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Transformed test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_test.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


#### Transformed test dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [15]:
train_df, _ = train_binary_dataset.convert_to_dataframe()
test_df, _ = test_binary_dataset.convert_to_dataframe()

In [16]:
train_df.head(10)

,cat__Mjob_at_home,cat__Mjob_health,cat__Mjob_other,cat__Mjob_services,cat__Mjob_teacher,cat__Fjob_at_home,cat__Fjob_health,cat__Fjob_other,cat__Fjob_services,cat__Fjob_teacher,...,num__failures,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,sex_binary,G3
17,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.462007,1.215287,-0.249043,-1.016419,-0.557247,-1.013096,0.322877,-0.221442,0.0,1.0
165,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.894524,0.050637,1.714836,-1.016419,-0.557247,-1.013096,-1.080495,1.159755,1.0,1.0
24,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.462007,0.050637,-0.249043,-1.016419,-0.557247,-1.013096,1.024563,-0.451641,0.0,0.0
126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.462007,1.215287,-0.249043,-1.016419,-0.557247,-1.013096,-1.782181,-0.681841,0.0,1.0
346,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.462007,1.215287,-0.249043,-1.016419,-0.557247,-0.247029,0.322877,0.354057,1.0,1.0
176,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.462007,-1.114013,0.732897,0.759098,-0.557247,1.285105,1.024563,-0.451641,0.0,1.0
275,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.462007,0.050637,0.732897,0.759098,0.565382,0.519038,1.024563,0.008758,0.0,1.0
246,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.462007,1.215287,-1.230982,-1.016419,-0.557247,-1.013096,-1.080495,-0.221442,1.0,1.0
239,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.894524,1.215287,1.714836,0.759098,1.688010,2.051172,-1.080495,-0.681841,1.0,0.0
153,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.607587,0.050637,1.714836,0.759098,-0.557247,-1.013096,0.322877,-0.681841,1.0,0.0


In [17]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)

In [18]:
train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

In [19]:
test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

In [20]:
rfc = rfc.fit(train_X, train_y)

In [21]:
y_pred = rfc.predict(test_X)

In [22]:
report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         0.0       0.81      0.28      0.42        46
         1.0       0.68      0.96      0.80        73

    accuracy                           0.70       119
   macro avg       0.75      0.62      0.61       119
weighted avg       0.73      0.70      0.65       119



In [23]:
train_df, _ = dataset_transf_train.convert_to_dataframe()
test_df, _ = dataset_transf_test.convert_to_dataframe()

In [24]:
train_df.head(10)

,cat__Mjob_at_home,cat__Mjob_health,cat__Mjob_other,cat__Mjob_services,cat__Mjob_teacher,cat__Fjob_at_home,cat__Fjob_health,cat__Fjob_other,cat__Fjob_services,cat__Fjob_teacher,...,num__failures,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,sex_binary,G3
17,0.282101,0.248208,0.484390,0.532686,0.400076,0.438113,0.156397,0.726901,0.607373,0.506497,...,0.390084,0.743162,0.335698,0.414230,0.522159,0.552276,0.514006,0.608915,0.0,1.0
165,0.249704,0.239742,0.447924,0.605743,0.371744,0.501155,0.164897,0.692198,0.632031,0.516646,...,0.430889,0.773674,0.377180,0.423218,0.477969,0.591852,0.483389,0.612473,1.0,1.0
24,0.286787,0.277285,0.510560,0.555359,0.376523,0.423796,0.193318,0.630097,0.549982,0.525848,...,0.334830,0.744751,0.268252,0.396082,0.552517,0.595887,0.549265,0.588588,0.0,1.0
126,0.283425,0.263831,0.503686,0.543200,0.399975,0.430144,0.173585,0.688904,0.569008,0.506249,...,0.359209,0.745229,0.306384,0.436113,0.559446,0.567496,0.504330,0.589389,0.0,1.0
346,0.318287,0.267461,0.513688,0.529047,0.397335,0.413002,0.181395,0.686483,0.561874,0.508954,...,0.329536,0.736252,0.291603,0.420168,0.588532,0.568906,0.517000,0.577047,1.0,1.0
176,0.282252,0.255658,0.481118,0.585136,0.384880,0.468259,0.179183,0.680416,0.588223,0.508834,...,0.376986,0.762485,0.340344,0.447044,0.550275,0.589141,0.473185,0.582843,0.0,1.0
275,0.286671,0.260240,0.487202,0.584699,0.383513,0.462902,0.184416,0.669277,0.578193,0.510370,...,0.365513,0.761147,0.329208,0.446071,0.560322,0.593267,0.477210,0.578044,0.0,1.0
246,0.267992,0.263812,0.508562,0.521976,0.409976,0.416502,0.163040,0.702821,0.569001,0.503836,...,0.368557,0.738356,0.298512,0.431070,0.543138,0.550714,0.520457,0.603434,1.0,1.0
239,0.308950,0.258256,0.491667,0.596990,0.397620,0.473029,0.188614,0.689607,0.568675,0.492481,...,0.351665,0.767680,0.352486,0.505402,0.619021,0.591391,0.409478,0.547248,1.0,1.0
153,0.262591,0.243234,0.459737,0.594076,0.381572,0.488665,0.166380,0.699859,0.618974,0.509857,...,0.414230,0.768845,0.369878,0.439144,0.507523,0.584657,0.471818,0.601322,1.0,1.0


In [25]:
test_df.head(10)

,cat__Mjob_at_home,cat__Mjob_health,cat__Mjob_other,cat__Mjob_services,cat__Mjob_teacher,cat__Fjob_at_home,cat__Fjob_health,cat__Fjob_other,cat__Fjob_services,cat__Fjob_teacher,...,num__failures,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,sex_binary,G3
78,0.255103,0.240229,0.458210,0.576314,0.384179,0.479387,0.156957,0.712790,0.628966,0.511989,...,0.425016,0.762400,0.366995,0.419080,0.483931,0.572562,0.495721,0.616989,1.0,1.0
371,0.284422,0.254031,0.481565,0.573242,0.386724,0.461068,0.174813,0.689389,0.593217,0.509517,...,0.378610,0.757735,0.338354,0.435808,0.543480,0.581992,0.485394,0.588732,1.0,1.0
248,0.290882,0.271316,0.511648,0.528820,0.388552,0.410683,0.180058,0.664009,0.561522,0.520080,...,0.342124,0.735734,0.274145,0.391217,0.547390,0.573660,0.555698,0.597483,1.0,1.0
55,0.277791,0.265136,0.504119,0.567840,0.406397,0.447894,0.178923,0.686083,0.556985,0.495552,...,0.358886,0.756976,0.321671,0.485969,0.587986,0.574810,0.452647,0.571043,0.0,1.0
390,0.310056,0.258612,0.484527,0.595020,0.378187,0.469811,0.191686,0.668112,0.580605,0.510143,...,0.353438,0.763813,0.338153,0.450312,0.582537,0.602267,0.461525,0.563557,1.0,1.0
223,0.346796,0.271048,0.525740,0.532521,0.407354,0.410276,0.190369,0.692770,0.543471,0.496932,...,0.301752,0.737158,0.294916,0.460792,0.648938,0.569928,0.472964,0.547068,1.0,1.0
42,0.271172,0.270933,0.505507,0.548645,0.387270,0.427226,0.180661,0.655630,0.560849,0.518502,...,0.355884,0.745265,0.283985,0.409384,0.538456,0.580782,0.537696,0.598076,1.0,1.0
234,0.268484,0.259258,0.494635,0.540415,0.396373,0.433663,0.165938,0.693776,0.584009,0.510923,...,0.378652,0.744997,0.311279,0.416687,0.525410,0.563776,0.523475,0.606460,1.0,1.0
316,0.286992,0.254375,0.477040,0.605730,0.384104,0.484259,0.184561,0.678132,0.586582,0.504072,...,0.375855,0.771240,0.355603,0.473112,0.569097,0.598257,0.439563,0.568892,0.0,1.0
116,0.275361,0.255923,0.495979,0.542954,0.408934,0.439139,0.163169,0.715557,0.582370,0.498013,...,0.378262,0.748113,0.329192,0.454304,0.553245,0.556024,0.482431,0.593116,1.0,1.0


In [31]:
train_df, _ = dataset_transf_train.convert_to_dataframe()
test_df, _ = dataset_transf_test.convert_to_dataframe()


rfc = RandomForestClassifier(random_state=42)

train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

rfc = rfc.fit(train_X, train_y)

y_pred = rfc.predict(test_X)

report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       119

    accuracy                           1.00       119
   macro avg       1.00      1.00      1.00       119
weighted avg       1.00      1.00      1.00       119



### Tune parameters for LFR

In [32]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=5, Ax=0.1, Ay=0.1, Az=1,
         verbose=1
        )
TR = TR.fit(train_binary_dataset, maxiter=5000, maxfun=5000)

dataset_transf_train = TR.transform(train_binary_dataset)
dataset_transf_test = TR.transform(test_binary_dataset)

train_df, _ = dataset_transf_train.convert_to_dataframe()
test_df, _ = dataset_transf_test.convert_to_dataframe()


rfc = RandomForestClassifier(random_state=42)

train_X = train_df.drop(['G3'], axis=1)
train_y = train_df['G3']

test_X = test_df.drop(['G3'], axis=1)
test_y = test_df['G3']

rfc = rfc.fit(train_X, train_y)

y_pred = rfc.predict(test_X)

report = classification_report(test_y, y_pred)
print("Classification Report:\n", report)

step: 0, loss: 0.2096919259442772, L_x: 1.356051463361366,  L_y: 0.61390326509763,  L_z: 0.012696453098377613
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          160     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.09692D-01    |proj g|=  1.08365D-02
step: 250, loss: 0.20865769056356984, L_x: 1.3543497905570838,  L_y: 0.6138755838033865,  L_z: 0.011835153127522775

At iterate    1    f=  2.08658D-01    |proj g|=  1.08922D-02
  ys=-3.344E-06  -gs= 1.033E-03 BFGS update SKIPPED
step: 500, loss: 0.20410230529342385, L_x: 1.345706705183642,  L_y: 0.6137634988811697,  L_z: 0.008155284886942666

At iterate    2    f=  2.04102D-01    |proj g|=  1.20085D-02
step: 750, loss: 0.22185574618190845, L_x: 1.2500383209930321,  L_y: 0.6140621450228668,  L_z: 0.03544569958031858

At iterate    3    f=  1.97524D-01    |proj g|=  9.98540D-03
step: 1000, loss: 0.20109266661164496, L_x: 1.296063749109992,  L_y: 0.613102

In [28]:
from aif360.algorithms.preprocessing import LFR
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# k - Number of prototypes
# Input recontruction quality - Ax
# Fairness constraint - Az
# Output prediction error - Ay
def find_optimal():

    k_values = [1, 2, 3, 4, 5]
    Az_values = [1.0, 5.0, 25.0, 50.0]
    Ay_values = [1.0, 2.0, 3.0, 4.0, 10.0]
    Ax_values = [0.01, 0.1, 1.0, 10.0]

    best_accuracy = 0.0
    best_params = None

    for k in k_values:
        for Az in Az_values:
            for Ay in Ay_values:
                for Ax in Ax_values:
                    lfr = LFR(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                              k=k, Az=Az, Ay=Ay, Ax=Ax)

                    lfr = lfr.fit(train_binary_dataset)

                    dataset_transf_train = lfr.transform(train_binary_dataset)

                    train_df, _ = dataset_transf_train.convert_to_dataframe()

                    train_X = train_df.drop(['G3'], axis=1)
                    train_y = train_df['G3']

                    rfc = RandomForestClassifier(random_state=42)

                    rfc.fit(train_X, train_y)

                    dataset_transf_test = lfr.transform(test_binary_dataset)
                    test_df, _ = dataset_transf_test.convert_to_dataframe()
                    test_X = test_df.drop(['G3'], axis=1)
                    test_y = test_df['G3']
                    y_pred = rfc.predict(test_X)

                    accuracy = accuracy_score(test_y, y_pred)

                    if accuracy > best_accuracy and accuracy != 1.0:
                        best_accuracy = accuracy
                        best_params = (k, Az, Ay, Ax)

                    params = (k, Az, Ay, Ax)
                    print("Parameters (k, Az, Ay, Ax):", params)
                    print("Accuracy:", accuracy)


    return best_accuracy, best_params

In [29]:
best_accuracy, best_params = find_optimal()
print("Best Parameters (k, Az, Ay, Ax):", best_params)
print("Best Accuracy:", best_accuracy)

Parameters (k, Az, Ay, Ax): (1, 1.0, 1.0, 0.01)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 1.0, 0.1)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 1.0, 1.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 1.0, 10.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 2.0, 0.01)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 2.0, 0.1)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 2.0, 1.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 2.0, 10.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 3.0, 0.01)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 3.0, 0.1)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 3.0, 1.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 3.0, 10.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 4.0, 0.01)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 4.0, 0.1)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 4.0, 1.0)
Accuracy: 1.0
Parameters (k, Az, Ay, Ax): (1, 1.0, 4.0, 10.0)
Accuracy: 1.0
Parameters (k, A

KeyboardInterrupt: 